In [1]:
from models import *
from jepa import JEPA
from datasets import H5Dataset, SparseDataset
from trainer import Trainer

In [2]:
d1 = H5Dataset('train', 'multi')
d2 = H5Dataset('train', 'cite')

In [3]:
import pandas as pd

df1 = pd.read_hdf('data/train_cite_inputs.h5', start=1000, stop=2000)
df2 = pd.read_hdf('data/train_multi_targets.h5', start=1000, stop=2000)

rna1 = list(df1.keys())
rna2 = list(df2.keys())
df2.head()

gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
b31fa801888c,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,5.560683,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
0e2c18dfbf7e,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449378,4.449378,...,0.0,0.000000,0.000000,0.0,0.000000,5.136666,0.0,4.449378,0.000000,4.449378
352aba65831c,4.535933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.535933,...,0.0,5.627375,4.535933,0.0,4.535933,4.535933,0.0,0.000000,0.000000,4.535933
53b116cbaf60,3.859019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.541565,3.859019,...,0.0,0.000000,4.541565,0.0,0.000000,4.541565,0.0,0.000000,0.000000,0.000000
bb79553d92ea,4.181534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,4.181534,4.867014,0.0,4.181534,0.000000,0.0,4.181534,4.181534,0.000000


In [4]:
for i in range(len(rna1)):
    rna1[i] = rna1[i].split('_')[0]

count = 0
idxs_shared = []
idxs_not_shared = []
for i, s in enumerate(rna1):
    if s in rna2:
        idxs_shared.append(i)
    else:
        idxs_not_shared.append(i)

In [5]:
model = LinearCoder([18386, 3664], dropout=0.01, input_2d=False, days=[2])
count_parameters(model)

80805856

In [8]:
batch_size = 128
multi = d1.get_dataloader(batch_size)
cite = d2.get_dataloader(batch_size)
device = torch.device('cuda')
model.train().to(device)
optim = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
for (x, day), y in multi:
    optim.zero_grad()
    y = y.to(device)
    inputs, targets = y[:, idxs_shared], y[:, idxs_not_shared]
    out = model(inputs, [torch.tensor(2),] * batch_size)
    
    loss = F.mse_loss(out, targets)
    print(loss.item())
    loss.backward()
    optim.step()

481.1336975097656
69.7893295288086
288.26904296875
182.71786499023438
51.296058654785156
61.30269241333008
119.25382995605469
108.91669464111328
49.6721076965332
12.913922309875488
28.107383728027344
61.05895233154297
65.21923828125
40.998661041259766
13.460490226745605
6.199322700500488
18.001663208007812
27.915307998657227
22.043642044067383
8.861246109008789
5.799225330352783
7.084419250488281
4.288426876068115
6.590734958648682
3.511699914932251
5.621718883514404
3.513073205947876
4.808241844177246
3.263010025024414
3.6714444160461426
3.4507508277893066
2.987992763519287
3.611771583557129
2.593909978866577
3.3344204425811768
2.521364450454712
2.9738566875457764
2.5844411849975586
2.486013650894165
2.6499643325805664
2.2172036170959473
2.5241072177886963
2.113948345184326
2.3997490406036377
2.1546761989593506
2.282546281814575
2.0954434871673584
2.0233566761016846
2.0816762447357178
1.979690670967102
2.088170051574707
1.9303090572357178
2.0210485458374023
1.9058079719543457
2.001823

2.4890503883361816
2.315194845199585
1.9418705701828003
1.8288525342941284
2.089934825897217
2.4262757301330566
2.5062389373779297
2.3322298526763916
1.9976451396942139
1.8265964984893799
1.9467246532440186
2.160769462585449
2.2855374813079834
2.126659393310547
1.8829847574234009
1.8255258798599243
1.9051281213760376
1.8983421325683594
1.8524386882781982
1.8108296394348145
1.8659687042236328
1.8913286924362183
1.8290139436721802
1.8247488737106323
1.8797391653060913
1.972581386566162
1.956061601638794
1.8593621253967285
1.8294222354888916
1.8877042531967163
1.8911857604980469
1.8454960584640503
1.7957267761230469
1.8407933712005615
1.8163343667984009
1.8192687034606934
1.829115867614746
1.8006733655929565
1.8062412738800049
1.8273552656173706
1.8138006925582886
1.8233230113983154
1.848877191543579
1.8791141510009766
1.8751354217529297
1.855579137802124
1.8653225898742676
1.817650318145752
1.829240083694458
1.8543838262557983
1.8279949426651
1.819570541381836
1.8349761962890625
1.819664

In [11]:
model.eval()
val = H5Dataset('val', 'multi').get_dataloader(1)
for (x, day), y in val:
    y = y.to(device)
    inputs, targets = y[:, idxs_shared], y[:, idxs_not_shared]
    out = model(inputs, [torch.tensor(2),] * batch_size)
    
    loss = F.mse_loss(out, targets)
    print(out.squeeze()[:20])
    print(targets.squeeze()[:20])
    print(loss)
    

tensor([ 0.9411,  1.2949,  0.6009,  3.6289,  1.0904,  2.5381,  2.1024, -0.8175,
         2.6332,  2.4375,  1.4167, -0.8837,  2.2073, -1.0108,  1.1138,  2.1828,
        -1.0853,  1.4916, -0.9354, -1.0824], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 5.1311, 0.0000, 0.0000, 5.1311,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.8062, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 0.8953,  1.2650,  0.5159,  3.8400,  1.0427,  2.4912,  2.2449, -0.7427,
         2.6885,  2.4667,  1.3546, -0.8363,  2.3562, -0.9280,  1.0586,  2.3331,
        -1.0039,  1.6190, -0.9056, -1.0214], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')
tensor(2.6027, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0348,  1.3838,  0.4971,  3.3005,  1.0878,  2

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        5.5625, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(3.0080, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0006,  1.3619,  0.5868,  3.6094,  1.0883,  2.5616,  2.3593, -0.8200,
         2.7032,  2.4823,  1.3539, -0.8989,  2.4193, -0.9894,  1.1314,  2.4051,
        -1.0769,  1.7293, -0.9793, -1.1128], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 5.4062, 0.0000, 6.3198, 5.4062, 0.0000, 0.0000,
        4.7175, 0.0000, 0.0000, 5.4062, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.6311, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0116,  1.3790,  0.5753,  3.7572,  1.1052,  2.5035,  2.3645, -0.8176,
         2.7482,  2.4736,  1.3782, -0.9202,  2.4426, -0.9929,  1.1540,  2.4483,
        -1.0840,  1.7808, -1.0081, -1.1211], device='cuda:0',

tensor([0.0000, 0.0000, 0.0000, 4.3202, 0.0000, 0.0000, 0.0000, 0.0000, 4.3202,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 4.3202, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.5528, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0424,  1.4055,  0.5353,  3.5996,  1.1020,  2.5927,  2.6164, -0.8073,
         2.7339,  2.5248,  1.3646, -0.9195,  2.4762, -0.9895,  1.1581,  2.4517,
        -1.0877,  1.9168, -1.0077, -1.1375], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 4.4592,
        0.0000, 0.0000, 0.0000, 4.4592, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.5433, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0255,  1.3844,  0.4782,  3.6243,  1.0726,  2.5719,  2.7837, -0.7536,
         2.7515,  2.5409,  1.2936, -0.8756,  2.5272, -0.9388,  1.1183,  2.5485,
        -1.0344,  1.9801, -0.9758, -1.0899], device='cuda:0',

tensor([0.0000, 0.0000, 0.0000, 5.2238, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.6826, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.1809,  1.5276,  0.5413,  3.5143,  1.1595,  2.6947,  3.1259, -0.8619,
         2.8285,  2.5898,  1.4195, -0.9875,  2.5078, -1.0293,  1.2496,  2.5434,
        -1.1585,  2.2809, -1.1134, -1.2443], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 4.6196, 0.0000, 3.9362, 4.6196, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 4.6196, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.5062, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0275,  1.3505,  0.5524,  3.4930,  1.1094,  2.5986,  2.5339, -0.8082,
         2.6841,  2.4966,  1.3742, -0.8858,  2.2218, -1.0111,  1.1393,  2.3001,
        -1.1008,  1.6860, -0.9629, -1.1195], device='cuda:0',

tensor([0.0000, 0.0000, 0.0000, 5.3719, 0.0000, 4.6834, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.7616, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0070,  1.4026,  0.5509,  4.2160,  1.1682,  2.5504,  2.4477, -0.8409,
         2.8281,  2.5526,  1.6417, -0.9938,  2.3557, -1.0421,  1.2117,  2.2719,
        -1.1585,  1.9493, -1.0726, -1.1945], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 5.7770, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.7586, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0394,  1.4083,  0.5657,  3.6476,  1.1289,  2.4956,  2.3781, -0.8119,
         2.7225,  2.4798,  1.3494, -0.9431,  2.4902, -1.0203,  1.1728,  2.4873,
        -1.1043,  1.7666, -1.0150, -1.1313], device='cuda:0',

...

tensor([0.0000, 0.0000, 0.0000, 4.4452, 0.0000, 4.4452, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.5601, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 0.9803,  1.3549,  0.6164,  3.7575,  1.1263,  2.4552,  2.0227, -0.8343,
         2.6883,  2.4318,  1.4154, -0.9357,  2.3516, -1.0377,  1.1618,  2.3312,
        -1.1139,  1.5447, -0.9888, -1.1130], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 5.3297, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        6.0204, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 5.3297, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.9081, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 0.9455,  1.2969,  0.5226,  3.7133,  1.0561,  2.5408,  2.4435, -0.7659,
         2.6998,  2.4783,  1.3668, -0.8465,  2.2926, -0.9432,  1.0874,  2.3110,
        -1.0311,  1.7168, -0.9302, -1.0573], device='cuda:0',

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 4.5452, 4.5452, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.7599, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0088,  1.3855,  0.5297,  4.0436,  1.1193,  2.5457,  2.5785, -0.8082,
         2.8234,  2.5343,  1.5041, -0.9351,  2.4046, -0.9836,  1.1694,  2.3936,
        -1.0934,  1.9718, -1.0352, -1.1452], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 5.8847, 0.0000, 0.0000, 4.5067, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.5447, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0371,  1.3750,  0.5923,  3.6233,  1.1323,  2.5693,  2.4392, -0.8350,
         2.7160,  2.4864,  1.4147, -0.9227,  2.2739, -1.0315,  1.1639,  2.3333,
        -1.1228,  1.6998, -1.0035, -1.1458], device='cuda:0',

tensor([0.0000, 0.0000, 0.0000, 4.6745, 0.0000, 0.0000, 0.0000, 0.0000, 4.6745,
        4.6745, 0.0000, 4.6745, 4.6745, 0.0000, 0.0000, 4.6745, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.7816, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0395,  1.4011,  0.5748,  3.7199,  1.1168,  2.5363,  2.4734, -0.8256,
         2.7619,  2.4943,  1.3869, -0.9296,  2.4392, -1.0019,  1.1704,  2.4583,
        -1.0980,  1.8392, -1.0224, -1.1398], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([4.7301, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.7424, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0862,  1.4472,  0.5313,  3.6191,  1.1201,  2.6019,  2.7745, -0.8145,
         2.7798,  2.5495,  1.3556, -0.9399,  2.5475, -0.9944,  1.1805,  2.5562,
        -1.1009,  2.0336, -1.0416, -1.1631], device='cuda:0',

tensor([0.0000, 0.0000, 0.0000, 5.8663, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        4.4885, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 4.4885, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor(2.3625, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0549,  1.4168,  0.5359,  3.6874,  1.1108,  2.5868,  2.6883, -0.8108,
         2.7733,  2.5351,  1.3806, -0.9252,  2.4871, -0.9874,  1.1663,  2.4927,
        -1.0902,  1.9763, -1.0252, -1.1466], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0.0000, 0.0000, 0.0000, 5.4799, 4.3896, 4.3896, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        4.3896, 0.0000], device='cuda:0')
tensor(2.5109, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([ 1.0708,  1.4232,  0.5469,  3.8674,  1.1464,  2.6283,  2.7664, -0.8386,
         2.8219,  2.5618,  1.5229, -0.9553,  2.3390, -1.0162,  1.2009,  2.3616,
        -1.1363,  2.0309, -1.0634, -1.1949], device='cuda:0',

KeyboardInterrupt: 